In [ ]:
# Standard library imports
from itertools import product
import types

# Related third party imports
import holoviews as hv
# from ipyparallel import require
import kwant
import numpy as np
import pandas as pd
import sympy
import sympy.interactive

# Local imports
import funcs
from funcs import constants

%matplotlib inline
hv.notebook_extension()
sympy.interactive.init_printing('mathjax')

In [ ]:
from kwant.continuum.discretizer import discretize
import sympy
from sympy.physics.quantum import TensorProduct as kr

def discretized_hamiltonian(a, dim=3):
    sx, sy, sz = [sympy.physics.matrices.msigma(i) for i in range(1, 4)]
    s0 = sympy.eye(2)
    k_x, k_y, k_z = kwant.continuum.momentum_operators
    x, y, z = kwant.continuum.position_operators
    B_x, B_y, B_z, Delta, mu, alpha, g, mu_B, hbar, V = sympy.symbols(
        'B_x B_y B_z Delta mu alpha g mu_B hbar V', real=True)
    m_eff = sympy.symbols('m_eff', commutative=False)
    c, c_tunnel = sympy.symbols('c, c_tunnel')  # c should be (1e18 / constants.meV) if in nm and meV

    if dim == 1:
        k_y = k_z = 0
    elif dim == 2:
        k_z = 0

    k = sympy.sqrt(k_x**2 + k_y**2 + k_z**2)
    kin = (1 / 2) * hbar**2 * (k_x**2 + k_y**2 + k_z**2) / m_eff * c
    
    ham = ((kin - mu + V(x)) * kr(s0, sz) +
           alpha * (k_y * kr(sx, sz) - k_x * kr(sy, sz)) +
           0.5 * g * mu_B * (B_x * kr(sx, s0) + B_y * kr(sy, s0) + B_z * kr(sz, s0)) +
           Delta * kr(s0, sx))

    subs_sm = [(Delta, 0)]
    subs_sc = [(g, 0), (alpha, 0)]
    subs_interface = [(c, c * c_tunnel), (alpha, 0)]

    templ_sm = discretize(ham.subs(subs_sm), grid_spacing=a)
    templ_sc = discretize(ham.subs(subs_sc), grid_spacing=a)
    templ_interface = discretize(ham.subs(subs_interface), grid_spacing=a)

    return templ_sm, templ_sc, templ_interface


def get_cuts(syst, lat, x_left=0, x_right=1):
    l_cut = [lat(*pos) for pos in [s.tag for s in syst.sites()] if pos[0] == x_left]
    r_cut = [lat(*pos) for pos in [s.tag for s in syst.sites()] if pos[0] == x_right]
    return l_cut, r_cut


def add_vlead(syst, lat, l_cut, r_cut):
    dim = lat.norbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    return syst


def hopping_between_cuts(syst, r_cut, l_cut):
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    def hopping(syst, params):
        return syst.hamiltonian_submatrix(params=params,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return hopping


def lat_from_temp(template):
    return next(iter(template.sites())).family


def add_disorder_to_template(template):
    def onsite_dis(site, disorder, salt):
        s0 = np.eye(2)
        sz = np.array([[1, 0], [0, 1]])
        s0sz = np.kron(s0, sz)
        spin = holes = True
        mat = s0sz if spin and holes else s0 if spin else sz
        mat = np.array(mat).astype(complex)
        return disorder * (uniform(repr(site), repr(salt)) - .5) * mat

    for site, onsite in template.site_value_pairs():
        onsite = template[site]
        template[site] = combine(onsite, onsite_dis, operator.add, 1)

    return template


def phase(site1, site2, B_x, B_y, B_z, orbital, e, hbar):
    x, y, z = site1.tag
    vec = site2.tag - site1.tag
    lat = site1[0]
    a = np.max(lat.prim_vecs)  # lattice_contant
    A = [B_y * z - B_z * y, 0, B_x * y]
    A = np.dot(A, vec) * a**2 * 1e-18 * e / hbar
    phi = np.exp(-1j * A)
    if orbital:
        if lat.norbs == 2:  # No PH degrees of freedom
            return phi
        elif lat.norbs == 4:
            return np.array([phi, phi.conj(), phi, phi.conj()],
                            dtype='complex128')
    else:  # No orbital phase
        return 1


def apply_peierls_to_template(template):
    """Adds p.orbital argument to the hopping functions."""
    for (site1, site2), hop in template.hopping_value_pairs():
        lat = site1[0]
        a = np.max(lat.prim_vecs)
        template[site1, site2] = combine(hop, phase, operator.mul, 2)
    return template


def delta(site1, site2):
    """This is still being used in tunnel_hops. Should not depend on
    this to make the algo more robust."""
    return np.argmax(site2.pos - site1.pos)


def at_interface(site1, site2, shape1, shape2):
    return ((shape1[0](site1) and shape2[0](site2)) or
            (shape2[0](site1) and shape1[0](site2)))

# TUNNEL BARRIER NOT IN LEADS?

In [ ]:
import operator
from combine import combine
from funcs import cylinder_sector, square_sector

def make_3d_wire(a, L, r1, r2, phi, angle, L_sc, site_disorder, with_vlead,
                 with_leads, with_shell, shape):
    """Create a cylindrical 3D wire partially covered with a
    superconducting (SC) shell, but without superconductor in the
    scattering region of length L.

    Parameters
    ----------
    a : int
        Discretization constant in nm.
    L : int
        Length of wire (the scattering part without SC shell.) Should be bigger
        than 4 unit cells (4*a) to have the vleads in a region without a SC shell.
    r1 : int
        Radius of normal part of wire in nm.
    r2 : int
        Radius of superconductor in nm.
    phi : int
        Coverage angle of superconductor in degrees.
    angle : int
        Angle of tilting of superconductor from top in degrees.
    L_sc : int
        Number of unit cells that has a superconducting shell. If the system
        has infinite leads, set L_sc=a.
    site_disorder : bool
        When True, syst requires `disorder` and `salt` aguments.
    with_vlead : bool
        If True a SelfEnergyLead with zero energy is added to a slice of the system.
    with_leads : bool
        If True it appends infinite leads with superconducting shell.
    with_shell : bool
        Adds shell to the correct areas. If False no SC shell is added and
        only a cylindrical wire will be created.
    shape : str
        Either `circle` or `square` shaped cross section.

    Returns
    -------
    syst : kwant.builder.FiniteSystem
        The finilized kwant system.
    hopping : function
        Function that returns the hopping matrix between the two cross sections
        of where the SelfEnergyLead is attached.

    Examples
    --------
    This doesn't use default parameters because the variables need to be saved,
    to a file. So I create a dictionary that is passed to the function.

    >>> syst_params = dict(A_in_SC=True, a=10, angle=0, site_disorder=False,
    ...                    L=30, L_sc=10, phi=185, r1=50, r2=70,
    ...                    shape='square', with_leads=True,
    ...                    with_shell=True, with_vlead=True)
    >>> syst, hopping = make_3d_wire(**syst_params)

    """
    assert L_sc % a == 0
    assert L % a == 0

    templ_normal, templ_sc, templ_interface = map(
        apply_peierls_to_template, discretized_hamiltonian(a))
    lat = lat_from_temp(templ_normal)
    syst = kwant.Builder()
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))

    # The parts with a SC shell are not counted in the length L, so it's
    # modified as:
    L += 2*L_sc

    if shape == 'square':
        shape_function = square_sector
    elif shape == 'circle':
        shape_function = cylinder_sector
    else:
        raise(NotImplementedError('Only square or circle wire cross section allowed'))

    # Wire scattering region shapes
    shape_normal = shape_function(r_out=r1, angle=angle, L=L, a=a)
    # Superconductor slice in the beginning of the scattering region of L_sc
    # unit cells
    shape_sc_start = shape_function(
        r_out=r2, r_in=r1, phi=phi, angle=angle, L=L_sc, a=a)
    # Superconductor slice in the end of the scattering region of L_sc unit
    # cells
    shape_sc_end = shape_function(
        r_out=r2, r_in=r1, phi=phi, angle=angle, L0=L-L_sc, L=L, a=a)

    # Lead shapes
    shape_sc_lead = shape_function(
        r_out=r2, r_in=r1, phi=phi, angle=angle, L=-1, a=a)
    shape_normal_lead = shape_function(r_out=r1, angle=angle, L=-1, a=a)
    
    sites_normal = []
    sites_sc = []
    
    # Fill the normal part in the scattering region
    if site_disorder:
        sites_normal += syst.fill(add_disorder_to_template(templ_normal), *shape_normal)
    else:
        sites_normal += syst.fill(templ_normal, *shape_normal)
        
    # Fill in the infinite lead
    sites_normal += lead.fill(templ_normal, *shape_normal_lead)

    if with_shell:
        # Add the SC shell to the beginning and end slice of the scattering
        # region and to the lead.
        sites_sc += syst.fill(templ_sc, *shape_sc_start)
        sites_sc += syst.fill(templ_sc, *shape_sc_end)
        sites_sc += lead.fill(templ_sc, *shape_sc_lead)

    # Define left and right cut in wire in the middle of the wire, a region
    # without superconducting shell.
    cuts = get_cuts(syst, lat, L // (2*a) - 1, L // (2*a))
    
    # Sort the sites in both lists
    cuts = [sorted(cut, key=lambda s: s.pos[2] * 100000 + s.pos[1]) for cut in cuts]

    if with_vlead:
        syst = add_vlead(syst, lat, *cuts)

    if with_shell:
        # Adding a tunnel barrier between SM and SC
        tunnel_hops = {delta(s1, s2): hop for
                       (s1, s2), hop in templ_interface.hopping_value_pairs()}
        for (site1, site2), hop in syst.hopping_value_pairs():
            for shape2 in [shape_sc_start, shape_sc_end]:
                if at_interface(site1, site2, shape_normal, shape2):
                    syst[site1, site2] = tunnel_hops[delta(site1, site2)]
        for (site1, site2), hop in lead.hopping_value_pairs():
            if at_interface(site1, site2, shape_normal_lead, shape_sc_lead):
                syst[site1, site2] = tunnel_hops[delta(site1, site2)]

    if with_leads:
        syst.attach_lead(lead)
        syst.attach_lead(lead.reversed())

    syst = syst.finalized()    
    hopping = hopping_between_cuts(syst, *cuts)
    return syst, hopping

syst_pars = dict(a=8, angle=0, site_disorder=True, L=80, L_sc=8,
                 phi=135, r1=50, r2=70, shape='circle', with_leads=True,
                 with_shell=True, with_vlead=False)
syst, hopping = make_3d_wire(**syst_pars)

kwant.plot(syst)

# Simpler way of calculating the super current (but slightly slower)

In [ ]:
from funcs import matsubara_frequency

def null_G(syst, params, n):
    en = matsubara_frequency(n, params)
    gf = kwant.greens_function(syst, en, out_leads=[0], in_leads=[0],
                               check_hermiticity=False, params=params)
    return gf.data[::2, ::2]

def current_from_G_0(G_0s, H12, phase, params):
    t = H12 * np.exp(1j * phase)
    dim = t.shape[0]
    I = 0
    for G_0 in G_0s:
        V = np.zeros_like(G_0, dtype=complex)
        v = t - H12
        V[:dim, dim:] = v.T.conj()
        V[dim:, :dim] = v
        gf = np.linalg.solve(np.identity(2*dim) - G_0 @ V, G_0)
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * params['T'] * params['current_unit'] * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I

matsfreqs = 500
G_0s = [null_G(syst, params, n) for n in range(matsfreqs)]
H12 = hopping(syst, params=params)
phases = np.linspace(-np.pi, np.pi, 51)
I = [current_from_G_0(G_0s, H12, phase, params) for phase in phases]
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# 1D

In [ ]:
def make_1d_wire(a=10, L=400, L_sc=400):
    """Create a 1D semiconducting wire of length `L` with superconductors
    of length `L_sc` on its ends.

    Parameters
    ----------
    a : int
        Discretization constant in nm.
    L : int
        Length of wire (the scattering semi-conducting part) in nm.
    L_sc : int
        Length of superconducting ends in nm.

    Returns
    -------
    syst : kwant.builder.FiniteSystem
        The finilized kwant system.
    hopping : function
        Function that returns the hopping matrix between the two cross sections
        of where the SelfEnergyLead is attached.
    """
    tb_normal, tb_sc, _ = discretized_hamiltonian(a, dim=1)
    lat = lat_from_temp(tb_normal)
    syst = kwant.Builder()

    def shape(x_left, x_right):
        return lambda s: x_left <= s.pos[0] < x_right, (x_left//a,)

    syst.fill(tb_sc, *shape(-L_sc, 0))
    syst.fill(tb_normal, *shape(0, L))
    syst.fill(tb_sc, *shape(L, L+L_sc))

    cuts = get_cuts(syst, lat, L//2, L//2+1)
    syst = add_vlead(syst, lat, *cuts)

    lead = kwant.Builder(kwant.TranslationalSymmetry([a]))
    lead.fill(tb_sc, lambda x: True, (0,))
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())

    syst = syst.finalized()

    hopping = hopping_between_cuts(syst, *cuts)
    return syst, hopping

syst, hopping = make_1d_wire(a=1, L=100, L_sc=10)

params = dict(T=0.06, Delta=0.250, mu=15, k=constants.k,
              hbar=constants.hbar, m_eff=constants.m_eff, current_unit=constants.current_unit, c=constants.c, 
              B_x=0, B_y=0, B_z=0, g=0, mu_B=0, alpha=0, V=lambda x:0)
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([funcs.current_at_phase(syst, hopping, params, H_0_cache, phase, tol=0.01, max_frequencies=100)
              for phase in phases])

hv.Curve((phases, I)) 

# Test 2D system

In [ ]:
def make_2d_system(X=2, Y=2):
    ham = "(t * (k_x^2 + k_y^2) - mu) * sigma_z + Delta * sigma_x"
    template_lead = kwant.continuum.discretize(ham)
    template = kwant.continuum.discretize(ham.replace('Delta', '0'))
    syst = kwant.Builder()
    syst.fill(template, lambda s: 0 <= s.tag[0] < X and 0 <= s.tag[1] < Y, (0, 0))
    lat = lat_from_temp(template)

    # Add 0 self energy lead
    cuts = get_cuts(syst, lat)
    syst = add_vlead(syst, lat, *cuts)

    # Leads
    lead = kwant.Builder(kwant.TranslationalSymmetry((1, 0)))
    lead.fill(template_lead, lambda s: 0 <= s.pos[1] < Y, (0, 0))
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()

    hopping = hopping_between_cuts(syst, *cuts)
    return syst, hopping

syst, hopping = make_2d_system()
kwant.plot(syst)

params = dict(T=0.01/100, t=1, Delta=0.01, mu=2, k=1, current_unit=1)
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([funcs.current_at_phase(syst, hopping, params, H_0_cache, phase, tol=0.01, max_frequencies=200)
              for phase in phases])
I_analytical = 2*params['Delta']*0.5*np.sin(phases)/np.sqrt(1-np.sin(0.5*phases)**2)*np.tanh(params['Delta']/2/params['T']*np.sqrt(1-np.sin(0.5*phases)**2))
hv.Curve((phases, I)) * hv.Curve((phases, I_analytical))


# 3D test system

In [ ]:
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, mu: -mu * sz_
onsite_lead = lambda site, mu, delta: -mu * sz_ + delta * sx_
hops = lambda site1, site2, t: -t * sz_

def make_test_system(X, Y, Z):
    lat = kwant.lattice.general(np.eye(3), norbs=2)
    syst = kwant.Builder()
    syst[(lat(x, y, z) for x in range(X) for y in range(Y) for z in range(Z))] = onsite
    syst[lat.neighbors()] = hops

    cuts = get_cuts(syst, lat)
    syst = add_vlead(syst, lat, *cuts)

    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0, 0)))
    lead[(lat(0, y, z) for y in range(Y) for z in range(Z))] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    hopping = hopping_between_cuts(syst, *cuts)
    
    return syst, hopping

delta = 0.01
params = dict(mu=0., t=1., delta=delta, k=1, current_unit=1, T=delta/100)
syst, hopping = make_test_system(X=3, Y=3, Z=3)
kwant.plot(syst)
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([funcs.current_at_phase(syst, hopping, params, H_0_cache, phase, tol=0.01, max_frequencies=200)
              for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

# 3D system with Discretizer Hamiltonian, squared system

In [ ]:
def make_3d_beam_system(X, Y, Z, a=10, test_hamiltonian=True):
    if test_hamiltonian:
        ham = '(t * (k_x**2 + k_y**2 + k_z**2) - mu) * sigma_z + Delta * sigma_x'
        tb_normal = kwant.continuum.discretize(ham.replace('Delta', '0'))
        tb_sc = kwant.continuum.discretize(ham)
    else:
        tb_normal, tb_sc, *_ = discretized_hamiltonian(a)
    
    lat = lat_from_temp(tb_normal)
    syst = kwant.Builder()
    syst.fill(tb_normal, lambda s: 0 <= s.pos[0] < X and 0 <= s.pos[1] < Y and 0 <= s.pos[2] < Z, (0, 0, 0))
    
    cuts = get_cuts(syst, lat)
    syst = add_vlead(syst, lat, *cuts)

    lead = kwant.Builder(kwant.TranslationalSymmetry((a, 0, 0)))
    lead.fill(tb_sc, lambda s: 0 <= s.pos[1] < Y and 0 <= s.pos[2] < Z, (0, 0, 0))

    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    syst = syst.finalized()
    hopping = hopping_between_cuts(syst, *cuts)

    return syst, hopping


syst, hopping = make_3d_beam_system(X=5, Y=3, Z=3, a=1, test_hamiltonian=True)
params = dict(T=0.01, Delta=1, mu=2, t=1, k=1, current_unit=1, c=1)
kwant.plotter.bands(syst.leads[2], params=params)
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([funcs.current_at_phase(syst, hopping, params, H_0_cache, phase, tol=0.01, max_frequencies=1000)
              for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

In [ ]:
syst, hopping = make_3d_beam_system(X=20, Y=30, Z=30, a=10, test_hamiltonian=False)
params = dict(T=1, Delta=0.250, mu=30, k=constants.k,
              hbar=constants.hbar, m_eff=constants.m_eff, current_unit=constants.current_unit, c=constants.c, 
              B_x=0, B_y=0, B_z=0, g=0, mu_B=0, alpha=0, V=lambda x:0)
kwant.plot(syst)
kwant.plotter.bands(syst.leads[2], params=params)
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([funcs.current_at_phase(syst, hopping, params, H_0_cache, phase, tol=0.01, max_frequencies=500)
              for phase in phases])
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# 1D Supplementary figure

In [ ]:
from hpc05 import Client
client = Client(profile='pbs', timeout=10)

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
len(dview)

In [ ]:
# next two lines because of https://github.com/ipython/ipyparallel/issues/205
%px import sys; sys.path.append('/home/bnijholt/Dropbox/Work/nanowire_current/')
%px import funcs, types, kwant

In [ ]:
syst_pars = dict(a=10, L=50, L_sc=1)
ham_pars = dict(B_y=0, B_z=0, Delta=0.25, g=50, mu_B=constants.mu_B, t=constants.t, V=lambda x: 0)
T = 1
alphas = np.linspace(0, 100, 200)
B_xs = np.linspace(0, 1.5, 200)
mus = [0.1, 0.3 ,1, 3, 10, 30]
vals = list(product(alphas, B_xs, mus))

@require('kwant', 'types', 'funcs')
def fun(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    syst, hopping = funcs.make_1d_wire(**syst_pars)
    p = types.SimpleNamespace(**ham_pars, alpha=x[0], B_x=x[1], mu=x[2])
    return funcs.I_c(syst, hopping, p, T=T)

current_phase = lview.map_async(fun, vals)

current_phase.wait_interactive()

current_phase = current_phase.result()

df = pd.concat((pd.DataFrame(vals, columns=['alpha', 'B_x', 'mu']), pd.DataFrame(current_phase)), axis=1)
df = df.assign(**syst_pars, **ham_pars, T=T).assign(**constants.__dict__)
df = df.assign(git_hash=funcs.get_git_revision_hash())
df.to_hdf('1d_alpha_vs_B_x.hdf', 'all_data', mode='w')

In [ ]:
def Image(group, x, y, z):
    bounds = (df[x].min(), df[y].min(), df[x].max(), df[y].max())
    table = group.pivot_table(index=[x], columns=[y], values=[z])
    return hv.Image(np.rot90(table), kdims=[x, y], vdims=[z], bounds=bounds)

In [ ]:
%%opts Image (cmap='viridis') Layout {+axiswise}
df = pd.read_hdf('data/1d_alpha_vs_B_x.hdf')
ims = [(mu, Image(group, 'B_x', 'alpha', 'current_c')) for mu, group in df.groupby('mu')]
hm = hv.HoloMap(ims, kdims=['mu'])
hm.layout('mu').cols(3)